In [4]:
import pickle
import os


data = pickle.load(open('/home/msun415/SynTreeNet/results/viz/top_1000/skeletons-top-1000-valid.pkl', 'rb'))
inds = [0,1,2,3,15,20,22,32,47,60,61,62,64,65,68,72,96,104,156,245,292,295,525,540,812,958,1107,2024,2252,6180]
sts = [list(data)[ind] for ind in inds]
smiles = []
for st in sts:
    for st in data[st]:
        assert len(st.reactions) <= 4
        smiles.append(st.root.smiles)

smiles[:10]

['CCNS(=O)(=O)c1cc([N+](=O)[O-])ccc1C1=NC(c2ccc(N=C=O)cc2F)=NN1',
 'COC(=O)c1ccc(NC(=O)Nc2ccc(C#N)nc2)c(O)c1',
 'Nc1c(Cl)cc(F)cc1C1=NC(c2cc([N+](=O)[O-])c(F)cc2F)=NN1',
 'CC(=O)N(c1nc(CNC(=S)Nc2cc(Br)cc(Cl)c2C(=O)O)cs1)c1c(C)cc(C)cc1Cl',
 'COC(=O)c1ccc(OCCNC(=O)CN=C=S)c(N)c1F',
 'CC(C)(C)c1ccc2c(c1)C(=O)NC(CN=C=S)=N2',
 'CC(C)Nc1ccc(Cl)cc1NS(=O)(=O)Cc1cc(F)ccc1[N+](=O)[O-]',
 'CCCn1c(-c2cc(Cl)cc(Cl)c2[N+](=O)[O-])nc2ccccc21',
 'COCCOc1cc(NS(=O)(=O)Cc2cc(F)ccc2[N+](=O)[O-])c(C(=O)O)cc1OC',
 'Cc1ccc(N=C=O)c(C2=NNC(Cc3ccc(N=C=S)cc3)=N2)c1']

In [7]:
len(smiles)
with open('/home/msun415/SynTreeNet/data/assets/molecules/depth-5.txt', 'w+') as f:
    for smile in smiles:
        f.write(f"{smile}\n")
